In [ ]:
import os
import random
import pandas as pd

from cdhf.data import Data
from tqdm.notebook import tqdm as log_progress
from utils.helpers import execute, save_pckl, create_init_dataframe
from utils.power_functions import PowerFuncScore

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
random.seed(42)
random_sample = random.sample(range(10, 300), 1)
data = Data("../input/mmdata.json")
data.load_all()

In [ ]:
df = create_init_dataframe(data)

In [ ]:
df_evals = pd.DataFrame()

feature_evals = {}

feature_path = "Processed/Ego"
feature_files = [f for f in os.listdir(feature_path) if os.path.isfile(os.path.join(feature_path, f))]
power_function = PowerFuncScore()

for feature_file in log_progress(feature_files):                
    feature_df = pd.read_pickle(os.path.join(feature_path, feature_file)).rename(columns={"userid":"user_id", "channelid":"channel_id"})    

    rec_df = pd.merge(df, feature_df, on=['user_id','channel_id'], how="outer").fillna(0)    
    rec_df.loc[:,'score'] = rec_df['score'].apply(lambda x: x*1000)
    feature_evals[feature_file] = execute(rec_df[["user_id", "channel_id", "score", "u_id", "c_id"]], random_sample) 
    path = f"Evaulation/Complex/Ego/{feature_file}" 
    save_pckl(path, feature_evals[feature_file])           

In [ ]:
for key in feature_evals:      
    eval_frame = feature_evals[key]    
    display(eval_frame)  